In [ ]:
!pip install reachability

In [ ]:
import requests
import time
import pickle
from multiprocessing import Pool
from urllib.parse import urlencode
from reachability import Reachability
from tqdm import tqdm

def get_docket_id(args):
    cluster_id, api_key, url, headers = args
    params = {
        "clusters": cluster_id,
        "fields": "id,clusters"
    }

    while True:
        try:
            newurl = f"{url}?{urlencode(params, safe=',')}"
            response = requests.get(newurl, headers=headers)
            response.raise_for_status()
            data = response.json()
            if data["count"] > 0:
                docket_id = data["results"][0]["id"]
                return [cluster_id, docket_id]
            else:
                print(f"No docket found for cluster ID: {cluster_id}")
                return [cluster_id, None]
        except requests.exceptions.HTTPError as err:
            # If the request fails, wait 3 minutes and try again with the same parameters
            print(err)
            time.sleep(180)
            continue
        except requests.exceptions.ConnectionError as err:
            reachable = Reachability(timeout=None)
            while not reachable.is_online():
                print("Waiting for internet connection...")
                time.sleep(120)
            continue
        except Exception as err:
            print("Other error: ", err)
            return [cluster_id, None]

def get_docket_ids(cluster_ids, api_key, url, headers):
    args_list = [(cluster_id, api_key, url, headers) for cluster_id in cluster_ids]
    with Pool() as pool:
        results = list(tqdm(pool.imap(get_docket_id, args_list), total=len(cluster_ids), desc="Retrieving docket IDs"))
    return results

cluster_ids = [4579237,4579238,4581379,4581380,4581219,4581524,4581525,4583334,4583335,4588564,4588565,4591645,4591646,4591917,4591916,4593882,4594419,4594417,4594558,4594559,4595877,4594418,4595874,4595899,4595875,4595876,4595943,4595944,4600777,4601079,4601080,4601078,4601391,4603642,4603641,4604048,4604808,4609491,4605151,4605632,4605633,4605634,4609490,4612906,4614408,4618955,4618953,4618954,4618958,4618951,4618956,4618957,4618952,4621280,4621276,4621277,4621278,4621279,4623613,4623615,4623614,4624917,4625565,4625689,4625690,4625567,4625566,4625564,4627816,4627817,4627818,4627815,4630088,4630267,4630268,4630087,4630085,4630086,4631414,4631465,4631442,4631394,4631842,4631844,4631843,4631841,4632240,4632239,4632235,4632238,4632237,4632236,4632952,4632951,4632953,4633470,4633469,4633471,4635093,4635094,4643441,4639918,4641200,4643440,4655294,4658631,4660210,4664354,4671164,4675729,4679337,4681447,4681446,4681445,4685017,4685182,4685183,4685522,4685881,4695147,4713897,4721458,4729279,4729476,4729477,4729478,4729479,4729777,4729776,4729774,4729775,4730118,4730117,4730163,4731815,4730119,4731524,4738308,4731814,4738307,4738312,4738309,4738313,4738310,4740294,4742384,4742387,4742383,4742386,4742385,4746631,4746633,4746632,4746634,4747781,4748672,4747779,4747780,4748671,4748669,4748670,4749015,4749013,4749014,4752099,4752098,4753847,4754172,4754738,4754635,4754907,4754908,4757575,4757654,4757655,4757653,4757656,4757658,4757657,4759029,4759028,4759315,4759316,4759389,4760994,4760995,4760996,4760997,4761897,4762680,4762679,4763596,4764331,4764453,4764451,4764602,4764603,4764452,4764769,4764770,4765904,4765903,4766378,4766379,4766668,4766667,4766664,4766665,4768429,4768213,4770835,4772304,4772303,4773160,4778372,4792659,4792660,4792854,4792661,4794488,4795892,4795626,4797910,4797911,4799054,4801280,4800350,4801238,4801279,4801278,4802500,4806328,4806488,4806487,4837664,4838087,4837662,4837663,4837665,4838506,4838845,4839214,4840575,4846628,4847128,4847590,4848299,4850243,4850393,4853224,4855229,4856049,4858037,4858038,4858987,4860995,4861817,4866333,4866335,4866332,4867543,4869840,4869841,4870761,4870760,4870762,4871034,4871033,4874692,4874693,4876197,4876196,4876198,4877180,4878495,4879015,4879014,4879558,4879669,4883692,4883693,4883739,4883740,4885593,4883694,4883696,4883695,4885592,4885590,4885591,4885589,4886808,4887957,4887956,4887959,4887971,4887972,4887958,4889469,4893490,4889470,4890458,4891453,4891452,4892569,4892570,4892571,4893489,4893421,4893420,4893422,4894055,4894283,4894284,4894122,4894054,4894407,4894056,4894053,4894406,4894911,4894912,4894913,4895264,4895263,4895711,4895902,4895710,4895709,4896548,4896549,4897039,4897040,4897041,4897042,4900456,4957281,4898459,4898455,4898458,4898460,4898858,4898456,4900455,4900457,4902410,4903381,4903413,4957280,4958317,5065494,5287260,5287261,5287262,5287263,5291689,5291995,5293626,5293793,5293794,5293795,5299726,5303420,5305358,5305639,5305640,5305924,5668854,6186439,6237263,6238073,6238238,6244457,6238489,6244456,6244621,6246300,6347183,6444142,6444930,6445865,6445866,6447545,6448163,6447544,6448059,6448162,6448058,6448085,6448086,6448818,6448615,6448616,6448935,6448774,6448934,6453005,6454239,6454238,6454963,6455147,6456420,6457347,6460297,6460296,6461471,6461470,6461473,6461472,6461828,6461474,6463787,6463720,6464817,6468902,6468901,6469025,6469026,6468900,6470913,6470914,6470912,6473471,6475113,6475112,6475114,6475794,6476369,6477112,6477115,6477116,6477114,6477113,6477117,6477854,6477853,6477908,6477852,6477906,6478733,6478769,6479297,6479293,6479295,6479296,6479298,6479294,6480695,6480698,6480697,6481357,6481358,6619005,6619573,6619002,6619004,6619003,6619572,6619823,6619822,6620338,6620339,6620340,6620341,6620343,6620344,6620342,6620345,6620346,6623244,6623460,6623243,6623459,7686272,8241283,8241404,8241403,8247829,8509977,8509974,8509973,8509976,8509975,8511111,9302602,9355164,9357760,9369767,9369769,9376936,9378410,9378409,9378411,9380169,9380168,9381468,9385046,9385353,9386887,9387163,9391546,9388838,9389987,9391545,9392094,9392441,9392442,9398337,9392443,9393410,9393584,9398333,9398336,9398334,9400078,9398335,9398332,9398952,9400076,9400072,9400074,9400077,9400442,9400073,9401905,9401904,9403373,9401906,9403370,9403371,9403372,9404969,9404967,9404970,9404968,9406658,9406660,9406659,9407187,9407188,9408681,9408682,9408684,9408683,9409091,9409094,9409093,9409092,9409349,9409606,9410341,9409607,9409605,9410339,9410340,9410338,9410832,9410833,9410834,9410837,9410785,9410830,9410831,9410835,9410787,9410786,9410836,9414932,9417457,9418243,9425202,9431584,9434091,9434366,9434377,9435309,9435902,9438686,9441732,9442948,9449042,9473279,9450797,9450798,9451452,9469207,9473277,9473278,9476532,9476531,9476533,9480638,9476534,9477033,9477032,9480637]

# Set up API parameters
api_key = "163cea228fb27936988d579ed72fe787848e1866"
url = "https://www.courtlistener.com/api/rest/v3/dockets/"
headers = {"Authorization": f"Token {api_key}"}

# Get docket IDs for the cluster IDs
docket_ids = get_docket_ids(cluster_ids, api_key, url, headers)

print("Cluster ID and Docket ID pairs:", docket_ids)

with open("/content/drive/MyDrive/docket_ids.pickle", "wb") as f:
    pickle.dump(docket_ids, f)

Retrieving docket IDs:  37%|███▋      | 222/596 [00:30<00:46,  8.00it/s]

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/dockets/?clusters=4797911&fields=id,clusters
502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/dockets/?clusters=6445866&fields=id,clusters


Retrieving docket IDs: 100%|██████████| 596/596 [04:16<00:00,  2.32it/s]

Cluster ID and Docket ID pairs: [[4579237, 8480011], [4579238, 8480012], [4581379, 8497101], [4581380, 8497102], [4581219, 8494454], [4581524, 8502087], [4581525, 8502088], [4583334, 13585331], [4583335, 13585332], [4588564, 14541540], [4588565, 14541541], [4591645, 14557919], [4591646, 14557920], [4591917, 14560168], [4591916, 14560167], [4593882, 14570112], [4594419, 14572838], [4594417, 14572836], [4594558, 14572978], [4594559, 14572979], [4595877, 14581867], [4594418, 14572837], [4595874, 14581864], [4595899, 14581998], [4595875, 14581865], [4595876, 14581866], [4595943, 14589880], [4595944, 14589881], [4600777, 14730349], [4601079, 14741675], [4601080, 14741676], [4601078, 14741674], [4601391, 14752731], [4603642, 14797494], [4603641, 14797493], [4604048, 14808987], [4604808, 14828646], [4609491, 14924519], [4605151, 14831872], [4605632, 14846759], [4605633, 14846760], [4605634, 14846761], [4609490, 14924518], [4612906, 14988889], [4614408, 15013727], [4618955, 15421412], [4618953